In [1]:
# Importação de bibliotecas para manipulação e visualização de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações para melhorar a visualização dos gráficos
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12

print("Ambiente configurado e bibliotecas importadas com sucesso!")

# Carregando os dados. Substitua 'seu_dataset.csv' pelo nome do seu arquivo.
try:
    df = pd.read_csv('../data/bank_transactions_data.csv')
    print("Dataset carregado com sucesso!")
    print(f"O dataset contém {df.shape[0]} linhas e {df.shape[1]} colunas.")
except FileNotFoundError:
    print("ERRO: Arquivo não encontrado. Verifique o caminho e nome do arquivo.")

Ambiente configurado e bibliotecas importadas com sucesso!
Dataset carregado com sucesso!
O dataset contém 2512 linhas e 16 colunas.


# Limpeza e Preparação dos Tipos de Dados

In [2]:
# Convertendo colunas de data para o formato datetime
# Essencial para cálculos de tempo.
print("Convertendo colunas de data para o formato datetime...")
try:
    df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
    df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])
    print("Conversão de datas concluída com sucesso.")
except Exception as e:
    print(f"Ocorreu um erro na conversão de datas: {e}")

# Verificando a mudança
df.info()

Convertendo colunas de data para o formato datetime...
Conversão de datas concluída com sucesso.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TransactionID            2512 non-null   object        
 1   AccountID                2512 non-null   object        
 2   TransactionAmount        2512 non-null   float64       
 3   TransactionDate          2512 non-null   datetime64[ns]
 4   TransactionType          2512 non-null   object        
 5   Location                 2512 non-null   object        
 6   DeviceID                 2512 non-null   object        
 7   IP Address               2512 non-null   object        
 8   MerchantID               2512 non-null   object        
 9   Channel                  2512 non-null   object        
 10  CustomerAge              2512 non-null   int64         
 11

 # Engenharia de Features (Feature Engineering)

In [3]:
# --- 1. Features de Tempo ---
# A que horas e em que dia a fraude ocorre? Fraudes noturnas são comuns.
print("\nCriando features de tempo...")
df['TransactionHour'] = df['TransactionDate'].dt.hour
df['DayOfWeek'] = df['TransactionDate'].dt.dayofweek # Segunda=0, Domingo=6

# Tempo desde a última transação (em segundos). Transações muito rápidas são suspeitas.
df['TimeSinceLastTransaction'] = (df['TransactionDate'] - df['PreviousTransactionDate']).dt.total_seconds()
print("Features de tempo criadas: TransactionHour, DayOfWeek, TimeSinceLastTransaction.")

# --- 2. Features de Risco Comportamental ---
# Qual a relação entre o valor da transação e o saldo? Gastar muito do saldo é estranho.
# Adicionamos um valor pequeno (epsilon) para evitar divisão por zero se o saldo for 0.
epsilon = 1e-6 
df['AmountToBalanceRatio'] = df['TransactionAmount'] / (df['AccountBalance'] + epsilon)
print("\nFeature de risco criada: AmountToBalanceRatio.")

# --- 3. (Opcional, Avançado) Features de Frequência ---
# Clientes que transacionam muito podem ter padrões diferentes.
print("\nCriando features de frequência...")
df['TransactionsPerAccount'] = df.groupby('AccountID')['TransactionID'].transform('count')
print("Feature de frequência criada: TransactionsPerAccount.")

# Visualizando as novas colunas criadas
print("\n--- Amostra do DataFrame com as Novas Features ---")
display(df.head())



Criando features de tempo...
Features de tempo criadas: TransactionHour, DayOfWeek, TimeSinceLastTransaction.

Feature de risco criada: AmountToBalanceRatio.

Criando features de frequência...
Feature de frequência criada: TransactionsPerAccount.

--- Amostra do DataFrame com as Novas Features ---


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,...,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionHour,DayOfWeek,TimeSinceLastTransaction,AmountToBalanceRatio,TransactionsPerAccount
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,...,Doctor,81,1,5112.21,2024-11-04 08:08:08,16,1,-49477134.0,0.002756,7
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,...,Doctor,141,1,13758.91,2024-11-04 08:09:35,16,1,-42823516.0,0.027345,7
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,...,Student,56,1,1122.35,2024-11-04 08:07:04,18,0,-41694656.0,0.112523,4
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,...,Student,25,1,8569.06,2024-11-04 08:09:06,16,4,-47403415.0,0.021531,8
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,...,Student,198,1,7429.40,2024-11-04 08:06:39,17,0,-33228915.0,0.001810,6


In [4]:
from sklearn.preprocessing import StandardScaler
import pickle
import os

print("\nIniciando pré-processamento final...")

# 1. Selecionar as colunas que serão usadas no modelo
# Incluímos as features originais e as que criamos. Excluímos IDs e datas brutas.
features_to_use = [
    'TransactionAmount', 'CustomerAge', 'TransactionDuration', 
    'LoginAttempts', 'AccountBalance', 'TransactionHour', 
    'DayOfWeek', 'TimeSinceLastTransaction', 'AmountToBalanceRatio',
    # Adicione aqui as colunas categóricas que serão transformadas
    'TransactionType', 'Channel', 'CustomerOccupation' 
]

df_model = df[features_to_use].copy()
print(f"\nColunas selecionadas para o modelo: {df_model.columns.tolist()}")

# 2. Codificação de Variáveis Categóricas (One-Hot Encoding)
# Transforma colunas de texto em colunas numéricas (0s e 1s)
print("\nAplicando One-Hot Encoding nas variáveis categóricas...")
df_model = pd.get_dummies(df_model, columns=['TransactionType', 'Channel', 'CustomerOccupation'], drop_first=True)
print("Novas colunas após One-Hot Encoding:", df_model.columns.tolist())

# 3. Escalonamento das Features Numéricas
# Coloca todas as variáveis na mesma escala para que nenhuma domine o modelo.
print("\nEscalonando todas as features...")
scaler = StandardScaler()
X_processed = scaler.fit_transform(df_model)

print("\nPré-processamento concluído!")
print(f"Shape do nosso dataset final para o modelo (X_processed): {X_processed.shape}")

# 4. Criando estrutura organizada de diretórios
print("\nCriando estrutura de diretórios...")

# Diretórios organizados
processed_dir = '../data/processed'
models_dir = '../models'
results_dir = '../results'
plots_dir = '../results/plots'
scores_dir = '../results/scores'

# Criar todos os diretórios
for directory in [processed_dir, models_dir, results_dir, plots_dir, scores_dir]:
    os.makedirs(directory, exist_ok=True)
    print(f"📁 Diretório criado/verificado: {directory}")

# 5. Salvando dados processados em data/processed/
print(f"\n💾 Salvando dados processados em: {processed_dir}/")

# Salvar os dados processados como numpy array
np.save(f'{processed_dir}/X_processed.npy', X_processed)
print(f"✅ Dados processados: {processed_dir}/X_processed.npy")

# Salvar as labels (variável target) se existir
if 'IsFraud' in df.columns:
    y = df['IsFraud'].values
    np.save(f'{processed_dir}/y.npy', y)
    print(f"✅ Labels: {processed_dir}/y.npy")

# Salvar o scaler para aplicar no futuro em novos dados
with open(f'{processed_dir}/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler: {processed_dir}/scaler.pkl")

# Salvar os nomes das colunas após o preprocessing
feature_names = df_model.columns.tolist()
with open(f'{processed_dir}/feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)
print(f"✅ Feature names: {processed_dir}/feature_names.pkl")

# Salvar um resumo dos dados processados
summary_info = {
    'original_shape': df.shape,
    'processed_shape': X_processed.shape,
    'feature_names': feature_names,
    'features_to_use': features_to_use,
    'preprocessing_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'categorical_columns_encoded': ['TransactionType', 'Channel', 'CustomerOccupation'],
    'scaler_type': 'StandardScaler'
}

with open(f'{processed_dir}/preprocessing_summary.pkl', 'wb') as f:
    pickle.dump(summary_info, f)
print(f"✅ Resumo do preprocessing: {processed_dir}/preprocessing_summary.pkl")

print(f"\n🎉 Estrutura organizada criada com sucesso!")
print(f"\n📊 Resumo do preprocessing:")
print(f"   - Dados originais: {df.shape}")
print(f"   - Dados processados: {X_processed.shape}")
print(f"   - Features finais: {len(feature_names)}")
print(f"   - Arquivos salvos em: {processed_dir}/")



Iniciando pré-processamento final...

Colunas selecionadas para o modelo: ['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'TransactionHour', 'DayOfWeek', 'TimeSinceLastTransaction', 'AmountToBalanceRatio', 'TransactionType', 'Channel', 'CustomerOccupation']

Aplicando One-Hot Encoding nas variáveis categóricas...
Novas colunas após One-Hot Encoding: ['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'TransactionHour', 'DayOfWeek', 'TimeSinceLastTransaction', 'AmountToBalanceRatio', 'TransactionType_Debit', 'Channel_Branch', 'Channel_Online', 'CustomerOccupation_Engineer', 'CustomerOccupation_Retired', 'CustomerOccupation_Student']

Escalonando todas as features...

Pré-processamento concluído!
Shape do nosso dataset final para o modelo (X_processed): (2512, 15)

Criando estrutura de diretórios...
📁 Diretório criado/verificado: ../data/processed
📁 Diretório criado/verificado: ../models
📁 Diretóri